# Import Module

In [13]:
import csv
import time
import sys
import os
import random

# other library
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# PyTorch library
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from tqdm.notebook import tqdm
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms
import random

# Fix random seed

## Download model


In [2]:
SEED = 5566 # Do not modify
use_gpu = torch.cuda.is_available()
device = torch.device("cuda" if use_gpu else "cpu")

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
random.seed(SEED)
np.random.seed(SEED)

# Dataset Download:

In [3]:
!gdown --id "1Rlt_mU-siC6AayAjNMXZ8N8fURWg5iLB" --output "trainX.npy"
!gdown --id "1v7i5rP2UQv_2ju8akh69M5VzEjRJV3tB" --output "visualization_X.npy"


/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Rlt_mU-siC6AayAjNMXZ8N8fURWg5iLB
To: /content/trainX.npy
100% 221M/221M [00:05<00:00, 41.8MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1v7i5rP2UQv_2ju8akh69M5VzEjRJV3tB
To: /content/visualization_X.npy
100% 15.4M/15.4M [00:00<00:00, 52.7MB/s]


# Set Hyper-parameters

In [41]:
#TODO: Modified the hyper-parameter
NUM_EPOCH = 50
BATCH_SIZE = 32
LATENT_DIM = 64
REDUCED_DIM = 16
NUM_ITER = 1000
REDUCED_METHOD = 'pca' # or 'tsne'
lr = 5e-4

In [42]:
MODEL_NAME = 'model.pth'
DATA_PATH = 'trainX.npy'

# Define Dataset

In [43]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data_path):
        self.total_img = torch.from_numpy(np.load(data_path)).float()
        self.total_img = self.total_img.permute(0, 3, 1, 2)
        self.total_img = self.total_img / 255

    def normalize(self, img):
        # ImageNet mean and std
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        img = TF.normalize(img, mean, std)
        return img

    def augment(self, img):
        # Create a transformation pipeline as a list
        transformations = [
            transforms.RandomHorizontalFlip(),  # Random horizontal flip
            transforms.RandomVerticalFlip(),    # Random vertical flip
            transforms.RandomRotation(20),      # Random rotation between -20 and 20 degrees
            transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),  # Randomly change the brightness, contrast, saturation and hue
        ]

        # Randomly apply each transformation with a 50% chance
        for transform in transformations:
            if random.random() > 0.5:
                img = transform(img)

        return img

    def __len__(self):
        return len(self.total_img)

    def __getitem__(self, index):
        img = self.total_img[index]
        img_aug = self.augment(img)

        img_aug = self.normalize(img_aug)
        img = self.normalize(img)
        return img_aug, img

# Define Model Architerchure

**Please finish this block to run this code!**


In [44]:
class Net(nn.Module):
    def __init__(self, image_channels=3, latent_dim=128):
        super(Net, self).__init__()
        self.latent_dim = latent_dim
        self.img_size = 32

        # Define the encoder
        # Input size: [batch_size, 3, 32, 32]
        self.encoder = nn.Sequential(
            nn.Conv2d(image_channels, 32, kernel_size=3, stride=2, padding=1),  # Output size: [batch_size, 32, 16, 16]
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),  # Output size: [batch_size, 64, 8, 8]
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),  # Output size: [batch_size, 128, 4, 4]
            nn.ReLU()
        )

        # Flatten and pass through the fully connected layer
        self.fc1 = nn.Linear(4 * 4 * 128, self.latent_dim)

        self.fc2 = nn.Linear(self.latent_dim, 4 * 4 * 128)

        # Define the decoder
        # Input size: [batch_size, 128, 4, 4]
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),  # Output size: [batch_size, 64, 8, 8]
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),  # Output size: [batch_size, 32, 16, 16]
            nn.ReLU(),
            nn.ConvTranspose2d(32, image_channels, kernel_size=4, stride=2, padding=1),  # Output size: [batch_size, 3, 32, 32]
            nn.Tanh()
        )

    def forward(self, x):
        feature_map = self.encoder(x)
        latent_vec = self.fc1(feature_map.view(feature_map.size(0), -1))
        feature_map2 = self.fc2(latent_vec).view(-1, 128, 4, 4)
        x_res = self.decoder(feature_map2)
        return latent_vec, x_res

# Define Training Process

In [45]:
def training(train, val, model, device, n_epoch, batch, save_name, lr):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print('=== start training, parameter total:%d, trainable:%d' % (total, trainable))
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr,
                             weight_decay=1e-5)
    best_loss = 100
    for epoch in range(n_epoch):
        total_loss = 0

        # training set
        model.train()
        idx = 0
        for image_aug, image in tqdm(train):
            image = image.to(device, dtype=torch.float)
            image_aug = image_aug.to(device, dtype=torch.float)
            _, reconsturct = model(image_aug)
            loss = criterion(reconsturct, image)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += (loss.item() / len(train))

            print('[Epoch %d | %d/%d] loss: %.4f' %
                 ((epoch+1), idx*batch, len(train)*batch, loss.item()), end='\r')
            idx += 1
        print("\n  Training  | Loss:%.4f " % total_loss)

        # validation set
        model.eval()
        total_loss = 0
        idx = 0
        with torch.no_grad():
            for image_aug, image in tqdm(val):
                image = image.to(device, dtype=torch.float)
                image_aug = image_aug.to(device, dtype=torch.float)
                _, reconstruct = model(image_aug)

                loss = criterion(reconstruct, image)
                total_loss += (loss.item() / len(val))
                idx += 1
            print(" Validation | Loss:%.4f " % total_loss)
        # save model
        if total_loss < best_loss:
                best_loss = total_loss
                print("saving model with loss %.4f...\n" % total_loss)
                torch.save(model.state_dict(), "%s" % save_name)

# Define Clustering Process

In [46]:
def clustering(model, device, loader, n_iter, reduced_method, reduced_dim, perplexity):
    assert reduced_method in ['pca', 'tsne', None]

    model.eval()
    latent_vec = torch.tensor([]).to(device, dtype=torch.float)
    for idx, (image_aug, image) in enumerate(loader):
        print("predict %d / %d" % (idx, len(loader)) , end='\r')
        image = image.to(device, dtype=torch.float)
        latent, r = model(image)
        latent_vec = torch.cat((latent_vec, latent), dim=0)

    latent_vec = latent_vec.cpu().detach().numpy()

    if reduced_method == 'tsne':
        tsne = TSNE(n_components=reduced_dim, verbose=1, method='exact', perplexity=perplexity, n_iter=n_iter)
        latent_vec = tsne.fit_transform(latent_vec)
    elif reduced_method == 'pca':
        pca = PCA(n_components=reduced_dim, copy=False, whiten=True, svd_solver='full')
        latent_vec = pca.fit_transform(latent_vec)

    kmeans = KMeans(n_clusters=2, random_state=0, max_iter=n_iter).fit(latent_vec)
    return kmeans.labels_

# Define write function

In [47]:
def write_output(predict_result, file_name='predict.csv'):
    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['id', 'label'])
        for i in range(len(predict_result)):
            writer.writerow([str(i), str(predict_result[i])])

# Main Process

In [48]:
# build dataset
dataset = Dataset(DATA_PATH)
print(len(dataset))

# Random split
train_set_size = int(len(dataset) * 0.85)
valid_set_size = len(dataset) - train_set_size
train_set, valid_set = data.random_split(dataset, [train_set_size, valid_set_size])

# set data loader
train_loader = data.DataLoader(train_set, batch_size=BATCH_SIZE, num_workers=1, shuffle=True)
valid_loader = data.DataLoader(valid_set, batch_size=BATCH_SIZE, num_workers=1, shuffle=False)

model = Net(latent_dim=LATENT_DIM).to(device)
print(model)
training(train_loader, valid_loader, model, device, NUM_EPOCH, BATCH_SIZE, MODEL_NAME, lr)



9000
Net(
  (encoder): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): ReLU()
  )
  (fc1): Linear(in_features=2048, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2048, bias=True)
  (decoder): Sequential(
    (0): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): ReLU()
    (4): ConvTranspose2d(32, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (5): Tanh()
  )
)
=== start training, parameter total:522979, trainable:522979


  0%|          | 0/240 [00:00<?, ?it/s]

[Epoch 1 | 7648/7680] loss: 0.4559
  Training  | Loss:0.7746 


  0%|          | 0/43 [00:00<?, ?it/s]

 Validation | Loss:0.7024 
saving model with loss 0.7024...



...

 Validation | Loss:0.4908 
saving model with loss 0.4908...



  0%|          | 0/240 [00:00<?, ?it/s]

[Epoch 50 | 7648/7680] loss: 0.6310
  Training  | Loss:0.4644 


  0%|          | 0/43 [00:00<?, ?it/s]

 Validation | Loss:0.4871 
saving model with loss 0.4871...



# Inference

In [49]:
test_loader = data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)
model.load_state_dict(torch.load(MODEL_NAME))
predicted = clustering(model, device, test_loader, NUM_ITER, reduced_method=REDUCED_METHOD, reduced_dim=REDUCED_DIM, perplexity=15)

# Invert the predictions: change 0s to 1s and 1s to 0s
predicted_inverted = np.logical_not(predicted).astype(int)
# Or alternatively: predicted_inverted = 1 - predicted

write_output(predicted_inverted, 'pred.csv')

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
